Megaline company is unsatisfy to descover many clients are using inherited plans. They want to develop a ML model to anlizye the customer's behavior to recommend one of the new plans: Smart and Ultra.

Access to behavioral data of subsribers who have already switched to the new plans.

- Crate a model that choose the correct plan
- Model developed accuracy threshold is 0.75

In [5]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# load file
df = pd.read_csv('users_behavior.csv')

# overview of dataset
print(df.head())
print()
print(df.info())
print()
print(df.describe())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246

We can see that the head() method help us to understand the information that holds the df, there is one characteristic that contains the plans ultra and smart, no need to divided and create the column, we do not found any Null value, so no need to clean the data, we also can see what is the mean value for every columns, specialy for the mb_used characteristic to know what is the accuracy value to search.

In [6]:
# segment the source data to train, validation and test 20% of data and train with 60%
df_train, df_temp = train_test_split(df, test_size=0.40, random_state=54321)
df_valid, df_test = train_test_split(df_temp, test_size=0.50, random_state=54321)


# Declare variables for characteristics and objective
features_train = df_train.drop(['mb_used', 'is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['mb_used', 'is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['mb_used', 'is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Arbol de decision
# Create loop for max_depth from 1 to 5 >
for depth in range(1,6):
    # Crete model & specify max_depth=depth
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)# < crea un modelo, especificar max_depth=depth >
    # Train the model
    model.fit(features_train, target_train)
    # find the predictions using the validation set
    predictions_valid =  model.predict(features_valid) # < encuentra las predicciones usando el conjunto de validación >

    print('\nmax_depth =', depth, ': ', end='')
    print('Validation Accuracy:', accuracy_score(target_valid, predictions_valid))

    test_predictions = model.predict(features_test)
    print('Test Accuracy:', accuracy_score(target_test, test_predictions))


max_depth = 1 : Validation Accuracy: 0.6967340590979783
Test Accuracy: 0.7744945567651633

max_depth = 2 : Validation Accuracy: 0.7262830482115086
Test Accuracy: 0.7947122861586314

max_depth = 3 : Validation Accuracy: 0.7216174183514774
Test Accuracy: 0.7931570762052877

max_depth = 4 : Validation Accuracy: 0.7387247278382582
Test Accuracy: 0.7962674961119751

max_depth = 5 : Validation Accuracy: 0.7278382581648523
Test Accuracy: 0.7822706065318819


In [7]:
# Sanity Check
# train the final model
final_model = DecisionTreeClassifier(max_depth=3, random_state=12345)
final_model.fit(features_train, target_train)

# create dataset with dummy data
sanity_data = pd.DataFrame({
    'calls': [150,25,67,30,140],
    'minutes': [900,130,500,230,1400],
    'messages' : [35,20,2,55,300],
    'mb_used' : [35000, 17000, 1000, 25000, 50000]
})

sanity_features = sanity_data.drop('mb_used', axis=1)

sanity_predictions = final_model.predict(sanity_features)

for i, pred in enumerate(sanity_predictions):
    plan = 'Ultra' if pred == 1 else 'Smart'

    print(f'Result {i+1} Prediction model: {plan}')


Result 1 Prediction model: Ultra
Result 2 Prediction model: Smart
Result 3 Prediction model: Smart
Result 4 Prediction model: Smart
Result 5 Prediction model: Smart


Conclusion:

I used different models to find out that the Decision Tree Classifier is the one that get the 0.75 requested by the project.

These were the outputs for the other models:

### Random Forest model: 

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 100): 0.7276119402985075

### Logistinc Regression model:

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7331950207468879
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7437810945273632

### Decision Tree Classifier model:

max_depth = 1 : Validation Accuracy: 0.747355320472931
Test Accuracy: 0.7442439327940261

max_depth = 2 : Validation Accuracy: 0.7697573117610454
Test Accuracy: 0.7672682016179216

max_depth = 3 : Validation Accuracy: 0.7753578095830741
Test Accuracy: 0.7741132545115121

max_depth = 4 : Validation Accuracy: 0.7797137523335408
Test Accuracy: 0.7710018668326073

max_depth = 5 : Validation Accuracy: 0.7809583074051026
Test Accuracy: 0.775980087118855

The last model learned reasonable patterns, and passed the Sanity Check by using dummy data to make sure the model is not learning obvious patterns.